Etapa 1: Implementção

kNN:
• Algoritmo para busca do vizinho mais próximo: Força Bruta e KD Tree;
• Métrica: Distância Euclidiana
• k: 1, 3 ou 5 para força bruta, k=1 para KD Tree
• Tipos de Features: Numéricas

In [ ]:
import collections

def minkowski(r=2):
    return lambda x,y: sum(abs(x - y)**r) ** (1/r)

def knn(observation, X_treino, Y_treino, k=3, dist=minkowski(2)):
    # dists = lista de pares (distancia, rotulo)
    dists = [ (dist(observation, x), y) for (x,y) in list(zip(X_treino, Y_treino)) ]
    dists.sort(key=lambda y: y[0])
    nns = dists[:k]
    # Extrai somente rotulos
    labels = [ n[1] for n in nns ]
    # Retorna lista (de tamanho 1, segundo o parametro) de elemento mais comuns, pares (rotulo, num de occorencias)
    classification = collections.Counter(labels).most_common(1)
    # Retorna apenas o rotulo desse unico elemento
    return classification[0][0]

In [ ]:
class KDTree:
    #def __init__(self):
    #    self.value = 0
    #    self.left = None
    #    self.right = None

    def __init__(self, X_train, dim=0):
        self.data = X_train.copy()

        # Divide no meio a dimensao correspondente
        self.median_ind = len(self.data)//2
        self.data.sort(axis=dim) 
        self.value = self.data[self.median_ind]

        self.max_dim = len(self.data[0])
        self.next_dim = (dim+1) % self.max_dim

        # Cria nodos filhos caso possivel
        self.left, self.right = None, None
        if self.median_ind > 0:
            self.left_split  = self.data[:self.median_ind]
            self.left  = KDTree(self.left_split,  dim = self.next_dim)

        if len(self.data)-self.median_ind-1 > 0:
            self.right_split = self.data[self.median_ind+1:]
            self.right  = KDTree(self.right_split,  dim = self.next_dim)

        print(self.left, self.value, self.right)
    


def knn_kdtree():
    pass

In [ ]:
temp = normalized_train[:4]
print(temp)
kd = KDTree(temp)
#kd = KDTree()
#kd.create_tree(temp)
#print(normalized_train[:1])

In [85]:
from numpy import random

def prior(y,dataset): # P(y = c)
    total = len(dataset)
    vals = dataset[y].unique()
    arr = []
    for i in range(len(vals)):
        prob = dataset[y].value_counts(vals[i])/total
        arr.append((vals[i],prob))
    return arr

def likelihood(y,dataset): # P(x | y = c)
    like = 1
    for i in np.arange(0,dataset.shape[1]):
        mu = np.mean(dataset[:,i])
        sigma = np.std(dataset[:,i])
        gauss = random.normal(loc=mu,scale=sigma,size=y.shape)
        #gauss = (1/np.sqrt(2*np.pi*(sigma**2)))*np.exp(-1*((y[i]-mu)**2)/(2*(sigma**2)))
        like = like * gauss
    return like
    
"""
def likelyhood(y, Z):
    def gaussian(x, mu, sig):
        return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))
    prob = 1
    for j in np.arange(0, Z.shape[1]):
        m = np.mean(Z[:,j])
        s = np.std(Z[:,j])      
        prob = prob*gaussian(y[j], m, s)
    return prob
"""

def evidence(y,dataset): # P(x)
    arr = prior(y,dataset)
    like = likelihood(y,dataset)
    ev = 0
    for tp in arr:
        ev = ev + (like*tp[1])
    return ev

In [86]:
def get_numerical_features(X_train):
    return [col for col in X_train.columns if X_train[col].dtypes != 'O']

In [90]:
def naive_bayes(dataset,y):
    like = 1
    ev = 1
    for i in get_numerical_features(y):
        like = like * likelihood(y,dataset)
        ev = evidence(i,dataset)
    return (prior(y,dataset)*like(y,dataset))/ev

Etapa 2: Treinamento e Inferência

Primeiramente deve-se organizar os dados e dividí-los entre dados de treino e dados de teste

In [72]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

data = pd.DataFrame(pd.read_csv('../penguins.csv'))

# Retirar valores invalidos
data.replace("Na",float("NaN"),inplace=True)
data.dropna(subset=["Culmen Length (mm)","Culmen Depth (mm)","Flipper Length (mm)","Body Mass (g)","Sex","Delta 15 N (o/oo)","Delta 13 C (o/oo)"],inplace=True)

# Shuffle dos dados não balanceados
#data = data.sample(frac=1)

# Separar atributo alvo dos restantes
y = data["Species"]
x = data.drop(columns="Species")

# Selecionar apenas colunas numericas relevantes
x = x.drop(columns=["studyName", "Sample Number", "Region", "Island", "Stage", "Individual ID", "Clutch Completion", "Date Egg", "Sex", "Delta 15 N (o/oo)", "Delta 13 C (o/oo)", "Comments"])

# Split dos dados
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.30, shuffle=True)

#split = 0.7
#ind_split = int(len(data) * split)
#
#x_treino = x[:ind_split]
#y_treino = y[:ind_split]
#
#x_teste = x[ind_split:]
#y_teste = y[ind_split:]

# Normalizar dados
scaler = StandardScaler()
normalized_train = scaler.fit_transform(x_train)

normalized_test = scaler.transform(x_test)

In [ ]:
print(list(zip(normalized_train, y_train.to_numpy())))

In [ ]:
# kd a tree???

# achei a tree
#aux = knn(normalized_test.iloc[[0]],x_teste,y_teste)
#print(np.append(x_teste, y_teste[..., np.newaxis], axis=1))

#umteste = normalized_test[0]
#y_hat = knn(umteste, normalized_train, y_train.to_numpy())
for (x,y) in zip(normalized_test, y_test.to_numpy()):
    y_hat = knn(x, normalized_train, y_train.to_numpy(), k=3)
    #print(y_hat, x)
    print(y_hat == y)

In [91]:
for (x,y) in zip(normalized_test, x_test.to_numpy()):
    x_hat = naive_bayes(normalized_train,x_train)
    print(x_hat)
#P = pd.DataFrame(data=np.zeros((x_test.shape[0], len(classes))), columns = classes) 
#for i in np.arange(0, len(classes)):
#    elements = tuple(np.where(y_train == classes[i]))
#    Z = x_train[elements,:][0]
#    for j in np.arange(0,x_test.shape[0]):
#        x = x_test[j,:]
#        pj = likelihood(x,Z)
#        P[classes[i]][j] = pj*len(elements)/x_train.shape[0]

KeyError: 0